In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [8]:
class Net(nn.Module):
    # def __init__(self):
    #     super(Net, self).__init__()
    #     self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
    #     self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    #     self.pool1 = nn.MaxPool2d(2, 2)
    #     self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    #     self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
    #     self.pool2 = nn.MaxPool2d(2, 2)
    #     self.conv5 = nn.Conv2d(256, 512, 3)
    #     self.conv6 = nn.Conv2d(512, 1024, 3)
    #     self.conv7 = nn.Conv2d(1024, 10, 3)

    # def forward(self, x):
    #     x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    #     x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    #     x = F.relu(self.conv6(F.relu(self.conv5(x))))
    #     x = F.relu(self.conv7(x))
    #     x = x.view(-1, 10)
    #     return F.log_softmax(x)
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 24, 24]             520
            Conv2d-2             [-1, 50, 8, 8]          25,050
            Linear-3                  [-1, 500]         400,500
            Linear-4                   [-1, 10]           5,010
Total params: 431,080
Trainable params: 431,080
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.12
Params size (MB): 1.64
Estimated Total Size (MB): 1.76
----------------------------------------------------------------


In [35]:


torch.manual_seed(1)
# batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=16, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=1000, shuffle=True, **kwargs)


In [36]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [37]:

# model = Net().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# for epoch in range(0, 20):
#     train(model, device, train_loader, optimizer, epoch)
#     test(model, device, test_loader)

loss=0.001139541040174663 batch_id=3749: 100%|██████████| 3750/3750 [00:39<00:00, 94.86it/s]



Test set: Average loss: 0.0559, Accuracy: 9804/10000 (98.0%)



loss=0.08882994949817657 batch_id=3749: 100%|██████████| 3750/3750 [00:39<00:00, 95.89it/s] 



Test set: Average loss: 0.0356, Accuracy: 9887/10000 (98.9%)



loss=0.01280993316322565 batch_id=3749: 100%|██████████| 3750/3750 [00:39<00:00, 95.80it/s] 


...


Test set: Average loss: 0.0265, Accuracy: 9929/10000 (99.3%)



loss=5.55802762391977e-06 batch_id=3749: 100%|██████████| 3750/3750 [00:39<00:00, 93.99it/s]



Test set: Average loss: 0.0261, Accuracy: 9929/10000 (99.3%)



loss=5.215404996761208e-08 batch_id=3749: 100%|██████████| 3750/3750 [00:40<00:00, 92.85it/s]



Test set: Average loss: 0.0262, Accuracy: 9931/10000 (99.3%)



In [38]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(0, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.16694259643554688 batch_id=3749: 100%|██████████| 3750/3750 [00:40<00:00, 91.90it/s]



Test set: Average loss: 0.0654, Accuracy: 9786/10000 (97.9%)



loss=0.06618165969848633 batch_id=3749: 100%|██████████| 3750/3750 [00:39<00:00, 94.50it/s]



Test set: Average loss: 0.0304, Accuracy: 9904/10000 (99.0%)



loss=0.0241028293967247 batch_id=3749: 100%|██████████| 3750/3750 [00:40<00:00, 93.60it/s] 


...


Test set: Average loss: 0.0260, Accuracy: 9930/10000 (99.3%)



loss=0.0004657179524656385 batch_id=3749: 100%|██████████| 3750/3750 [00:39<00:00, 93.96it/s] 



Test set: Average loss: 0.0277, Accuracy: 9922/10000 (99.2%)



loss=3.6357826047606068e-06 batch_id=3749: 100%|██████████| 3750/3750 [00:39<00:00, 95.11it/s] 



Test set: Average loss: 0.0291, Accuracy: 9926/10000 (99.3%)

